In [3]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas
import json
import numpy as np

modelloc = "iNlan20.xml" # input location of SBML model
fbaloc = "fluxes\\fbafluxes.json" # output location of fba solution fluxes
sampleloc = "fluxes\\fluxsamples.json" # output location of flux samples

### Run the basic model
This version of the model uses an iron hydrogenase (the base case). It is the default configuration of the model, both the bifurcating hydrogenase, and complex 1, 2 and ATP synthase modules are turned off. The default glucose feed is 1.5 mmol/gdw/h but can be changed by modifying "EX_glc__D_e".

In [4]:
model = cobra.io.read_sbml_model(modelloc)

fba_sol = model.optimize()
fba_sol.fluxes.to_json(fbaloc)

model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cys__L_e,EX_cys__L_e,0.01178,3,0.39%
glc__D_e,EX_glc__D_e,1.5,6,99.61%
h2o_e,EX_h2o_e,0.4995,0,0.00%
nh4_e,EX_nh4_e,0.3315,0,0.00%
pi_e,EX_pi_e,0.06543,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-1.194,2,32.04%
co2_e,EX_co2_e,-0.004537,1,0.06%
etoh_e,EX_etoh_e,-1.227,2,32.93%
for_e,EX_for_e,-2.606,1,34.97%


### Run the model using the bifurcating hydrogenase
This version of the model uses the bifurcating hydrogenase instead of the iron hydrogenase.

In [5]:
model = cobra.io.read_sbml_model(modelloc)

# turn the bifurcating hydrogenase on
rxn = model.reactions.get_by_id("HYDhbi")
rxn.lower_bound = 0
rxn.upper_bound = 1000

# turn the iron hydrogenase off
rxn = model.reactions.get_by_id("HYDhfe")
rxn.lower_bound = 0
rxn.upper_bound = 0

fba_sol = model.optimize()
fba_sol.fluxes.to_json(fbaloc)

model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cys__L_e,EX_cys__L_e,0.01353,3,0.45%
glc__D_e,EX_glc__D_e,1.5,6,99.55%
h2o_e,EX_h2o_e,1.126,0,0.00%
nh4_e,EX_nh4_e,0.3806,0,0.00%
pi_e,EX_pi_e,0.07513,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-1.923,2,53.26%
co2_e,EX_co2_e,-1.555,1,21.53%
etoh_e,EX_etoh_e,-0.4116,2,11.40%
for_e,EX_for_e,-0.9973,1,13.81%


### Run the model using complex 1 and 2, and the ATP synthase
This version of the model makes use of a proton motive force to generate ATP

In [6]:
model = cobra.io.read_sbml_model(modelloc)

# Activate the proton pumping machinery 
rxn = model.reactions.get_by_id("CMPL1h")
rxn.lower_bound = -1000
rxn.upper_bound = 1000

rxn = model.reactions.get_by_id("CMPL2h")
rxn.lower_bound = -1000
rxn.upper_bound = 1000

rxn = model.reactions.get_by_id("FUMh")
rxn.lower_bound = -1000
rxn.upper_bound = 1000

rxn = model.reactions.get_by_id("ATPShydr")
rxn.lower_bound = 0
rxn.upper_bound = 1000

fba_sol = model.optimize()
fba_sol.fluxes.to_json(fbaloc)

model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cys__L_e,EX_cys__L_e,0.01178,3,0.39%
glc__D_e,EX_glc__D_e,1.5,6,99.61%
h2o_e,EX_h2o_e,0.4995,0,0.00%
nh4_e,EX_nh4_e,0.3315,0,0.00%
pi_e,EX_pi_e,0.06543,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-1.194,2,32.04%
co2_e,EX_co2_e,-0.004537,1,0.06%
etoh_e,EX_etoh_e,-1.227,2,32.93%
for_e,EX_for_e,-2.606,1,34.97%


### Run base case model using lignocellulose as a substrate
Lignocellulose is a complex biopolymer composed of cellulose, hemicellulose and lignin. This model contains two reactions, `r_cellulase` and `r_hemicellulase` (associated with 233 and 310 genes respectively, of which 451 are unique, see `OmicsData\Cazymes` for the classification rules used to define hemicelluase and cellulase genes in the model). The input fluxes used here are arbitrary since it is very hard to measure the breakout products, and their fluxes into the gut fungus in vivo. However, these reactions qualitatively demonstrate that the model can grow on lignocellulose.

In [11]:
model = cobra.io.read_sbml_model(modelloc)

# Switch the glucose exchange off
glc_ex = model.reactions.get_by_id("EX_glc__D_e")
glc_ex.lower_bound = 0
glc_ex.upper_bound = 0


# Metabolize cellulose
cc_ex = model.reactions.get_by_id("EX_cellulose_e")
cc_ex.lower_bound = -0.5
cc_ex.upper_bound = -0.5

# Metabolize hemicellulose
hc_ex = model.reactions.get_by_id("EX_hemicellulose_e")
hc_ex.lower_bound = -0.25
hc_ex.upper_bound = -0.25



fba_sol = model.optimize()
fba_sol.fluxes.to_json(fbaloc)

model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cellulose_e,EX_cellulose_e,0.5,24,82.43%
cys__L_e,EX_cys__L_e,0.01949,3,0.40%
h2o_e,EX_h2o_e,1.6,0,0.00%
hemicellulose_e,EX_hemicellulose_e,0.25,10,17.17%
nh4_e,EX_nh4_e,0.5484,0,0.00%
pi_e,EX_pi_e,0.1082,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-1.91,2,32.00%
etoh_e,EX_etoh_e,-1.803,2,30.21%
for_e,EX_for_e,-3.865,1,32.37%


### Redox analysis
Evaluate the effect of the bifurcating hydrogenase on the regeneration of NAD.

In [5]:
model = cobra.io.read_sbml_model(modelloc)

pba_sol = cobra.flux_analysis.pfba(model)
model.metabolites.nad_c.summary() # Iron hydrogenase only

Percent,Flux,Reaction,Definition
0.60%,0.01527,AASAD2,L2aadp_c + atp_c + h_c + nadh_c --> L2aadp6sa_c + amp_c + nad_c + ppi_c
48.01%,1.227,ACALD,acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c
48.01%,1.227,ALCD2x,etoh_c + nad_c <=> acald_c + h_c + nadh_c
0.51%,0.01301,EAR100x,h_c + nadh_c + tdec2eACP_c --> dcaACP_c + nad_c
0.51%,0.01301,EAR120x,h_c + nadh_c + tddec2eACP_c --> ddcaACP_c + nad_c
0.51%,0.01301,EAR140x,h_c + nadh_c + tmrs2eACP_c --> myrsACP_c + nad_c
0.28%,0.007165,EAR160x,h_c + nadh_c + tpalm2eACP_c --> nad_c + palmACP_c
0.05%,0.001316,EAR180x,h_c + nadh_c + toctd2eACP_c --> nad_c + ocdcaACP_c
0.51%,0.01301,EAR40x,but2eACP_c + h_c + nadh_c --> butACP_c + nad_c
0.51%,0.01301,EAR60x,h_c + nadh_c + thex2eACP_c --> hexACP_c + nad_c


In [6]:
rxn = model.reactions.get_by_id("HYDhbi")
rxn.lower_bound = 0
rxn.upper_bound = 1000

rxn = model.reactions.get_by_id("HYDhfe")
rxn.lower_bound = 0
rxn.upper_bound = 0

pba_sol = cobra.flux_analysis.pfba(model)
model.metabolites.nad_c.summary() # Bifurcating hydrogenase 
#Note: MASh is the putative asparate malate shuttle that transports NAD between the cytosol and the hydrogenosome.

Percent,Flux,Reaction,Definition
0.70%,0.01753,AASAD2,L2aadp_c + atp_c + h_c + nadh_c --> L2aadp6sa_c + amp_c + nad_c + ppi_c
16.53%,0.4116,ACALD,acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c
16.53%,0.4116,ALCD2x,etoh_c + nad_c <=> acald_c + h_c + nadh_c
0.60%,0.01494,EAR100x,h_c + nadh_c + tdec2eACP_c --> dcaACP_c + nad_c
0.60%,0.01494,EAR120x,h_c + nadh_c + tddec2eACP_c --> ddcaACP_c + nad_c
0.60%,0.01494,EAR140x,h_c + nadh_c + tmrs2eACP_c --> myrsACP_c + nad_c
0.33%,0.008226,EAR160x,h_c + nadh_c + tpalm2eACP_c --> nad_c + palmACP_c
0.06%,0.001511,EAR180x,h_c + nadh_c + toctd2eACP_c --> nad_c + ocdcaACP_c
0.60%,0.01494,EAR40x,but2eACP_c + h_c + nadh_c --> butACP_c + nad_c
0.60%,0.01494,EAR60x,h_c + nadh_c + thex2eACP_c --> hexACP_c + nad_c


In [7]:
model.metabolites.nad_h.summary() # hydrogenosome

Percent,Flux,Reaction,Definition
100.00%,1.55,HYDhbi,2.0 fdxrd_h + 3.0 h_h + nadh_h --> 2.0 fdxox_h + 2.0 h2_h + nad_h
Percent,Flux,Reaction,Definition
100.00%,-1.55,MASh,h_c + nad_h + nadh_c <=> h_h + nad_c + nadh_h


### Flux variability analysis

In [8]:
model = cobra.io.read_sbml_model(modelloc)

fba_sol = model.optimize()
model.summary(fva=0.9)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
cys__L_e,EX_cys__L_e,0.01178,[0; 0.01178],3,0.39%
glc__D_e,EX_glc__D_e,1.5,[1.5; 1.5],6,99.61%
h2o_e,EX_h2o_e,0.4995,[0.07658; 0.5996],0,0.00%
nh4_e,EX_nh4_e,0.3315,[0.2984; 0.3391],0,0.00%
pi_e,EX_pi_e,0.06543,[0.05889; 0.06543],0,0.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ac_e,EX_ac_e,-1.194,[-1.492; -0.7013],2,32.04%
co2_e,EX_co2_e,-0.004537,[-1.751; 0],1,0.06%
etoh_e,EX_etoh_e,-1.227,[-1.777; -0.7313],2,32.93%
for_e,EX_for_e,-2.606,[-2.645; -0.8756],1,34.97%


### Sampling
Comment constraints as required.

In [18]:
model = cobra.io.read_sbml_model(modelloc)

## Experimentally measured fluxes of various metabolites (un)comment to further constrain the model
h2_ex = model.reactions.get_by_id("EX_h2_e")
h2_ex.lower_bound = 0.047483008
h2_ex.upper_bound = 0.189248895

h2_ex = model.reactions.get_by_id("EX_lac__D_e")
h2_ex.lower_bound = 0.716237324
h2_ex.upper_bound = 1.08919967

h2_ex = model.reactions.get_by_id("EX_for_e")
h2_ex.lower_bound = 1.087540278
h2_ex.upper_bound = 1.793056312

h2_ex = model.reactions.get_by_id("EX_etoh_e")
h2_ex.lower_bound = 0.472808257
h2_ex.upper_bound = 1.013514567

h2_ex = model.reactions.get_by_id("EX_ac_e")
h2_ex.lower_bound = 0.423987759
h2_ex.upper_bound = 0.711876893

h2_ex = model.reactions.get_by_id("EX_succ_e")
h2_ex.lower_bound = 0.02143
h2_ex.upper_bound = 0.04529

fba_sol = model.optimize()

mu = fba_sol.fluxes["Biomass"]
bm_rxn = model.reactions.get_by_id('Biomass')
mu_cons = model.problem.Constraint(bm_rxn.flux_expression, lb = 0.90*mu, ub=mu)
model.add_cons_vars(mu_cons)
s = cobra.sampling.sample(model, 2000, method="achr")
s.to_json(sampleloc)